<a href="https://colab.research.google.com/github/francotestori/aauba_02/blob/master/armado_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico 2

## Armado del dataset

In [1]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
!mkdir speechcommands
!tar -xf speech_commands_v0.01.tar.gz -C /content/speechcommands

--2020-06-23 14:14:23--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.125.128, 2404:6800:4008:c01::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.125.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz’

speech_commands_v0. 100%[===================>]   1.39G  68.8MB/s    in 26s     

2020-06-23 14:14:49 (55.3 MB/s) - ‘speech_commands_v0.01.tar.gz’ saved [1489096277/1489096277]



In [2]:
# Import de Librerías
import librosa
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import Audio
from librosa.display import specshow

In [3]:
# Busco y guardo en un mapa todos los audio files correspondientes a los digit_keys
digit_command_keys = [
  'one',
  'two',
  'three',
  'four',
  'five',
  'six',
  'seven',
  'eight',
  'nine'
]

audio_files = {}

for key in digit_command_keys:
  directory = f'speechcommands/{key}/*.wav'
  audio_files[key] = glob.glob(directory)

In [12]:
# Utilizamos la función propuesta en el TP 
# para hacer la extracción de los datos 
# correspondientes a cada audio file

def calculate_features(
    filename,
    n_mfcc=12,
    delta=True,
    deltadelta=True,
    energy=True, 
    summary_fn = [np.mean, np.std], 
    summary_names=['mean','std']):
  #Abro el archivo:
  x, sr = librosa.core.load(filename,sr=None)
  
  #Calculo MFCCs
  features = librosa.feature.mfcc(x,sr=sr,n_mfcc=n_mfcc)
  feat_names = ['mfcc_{}'.format(i) for i in range(n_mfcc)]
  #Calculo energia:
  if energy:
    energy = librosa.feature.rmse(x)
    features = np.concatenate([features,energy])
    feat_names = feat_names + ['energy']
  #Aplico media y desvio estandar por defecto
  summary_features = np.concatenate([fn(features,axis=1) for fn in summary_fn])
  
  #Lo mismo con los delta
  if delta:
    deltafeatures = np.diff(features)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltafeatures,axis=1) for fn in summary_fn])])
    d_names = ['d{}'.format(name) for name in feat_names]
  else:
    d_names = []

  #Y con los delta de segundo orden
  if deltadelta:
    deltadeltafeatures = np.diff(features,n=2)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltadeltafeatures,axis=1) for fn in summary_fn])]) 
    dd_names = ['dd{}'.format(name) for name in feat_names]
  else:
    dd_names = []

  feat_names = feat_names + d_names + dd_names
  feat_names = ['{}_{}'.format(feat,summary) for feat in feat_names for summary in summary_names]
  return summary_features, feat_names  

In [ ]:
audio_df = []

for key in audio_files:
  for audio in audio_files[key]:
    feat, names = calculate_features(audio)
    audio_row = dict(zip(names, feat))
    audio_row['digit'] = key

    audio_df.append(audio_row)
  
audio_df = pd.DataFrame(audio_df)

In [14]:
audio_df

,mfcc_0_mean,mfcc_0_std,mfcc_1_mean,mfcc_1_std,mfcc_2_mean,mfcc_2_std,mfcc_3_mean,mfcc_3_std,mfcc_4_mean,mfcc_4_std,mfcc_5_mean,mfcc_5_std,mfcc_6_mean,mfcc_6_std,mfcc_7_mean,mfcc_7_std,mfcc_8_mean,mfcc_8_std,mfcc_9_mean,mfcc_9_std,mfcc_10_mean,mfcc_10_std,mfcc_11_mean,mfcc_11_std,energy_mean,energy_std,dmfcc_0_mean,dmfcc_0_std,dmfcc_1_mean,dmfcc_1_std,dmfcc_2_mean,dmfcc_2_std,dmfcc_3_mean,dmfcc_3_std,dmfcc_4_mean,dmfcc_4_std,dmfcc_5_mean,dmfcc_5_std,dmfcc_6_mean,dmfcc_6_std,dmfcc_7_mean,dmfcc_7_std,dmfcc_8_mean,dmfcc_8_std,dmfcc_9_mean,dmfcc_9_std,dmfcc_10_mean,dmfcc_10_std,dmfcc_11_mean,dmfcc_11_std,denergy_mean,denergy_std,ddmfcc_0_mean,ddmfcc_0_std,ddmfcc_1_mean,ddmfcc_1_std,ddmfcc_2_mean,ddmfcc_2_std,ddmfcc_3_mean,ddmfcc_3_std,ddmfcc_4_mean,ddmfcc_4_std,ddmfcc_5_mean,ddmfcc_5_std,ddmfcc_6_mean,ddmfcc_6_std,ddmfcc_7_mean,ddmfcc_7_std,ddmfcc_8_mean,ddmfcc_8_std,ddmfcc_9_mean,ddmfcc_9_std,ddmfcc_10_mean,ddmfcc_10_std,ddmfcc_11_mean,ddmfcc_11_std,ddenergy_mean,ddenergy_std,digit
0,-645.621054,123.615676,56.655862,25.306103,-1.559808,0.350541,-15.113485,-10.303239,-17.813655,-2.959814,-10.568252,-11.304471,0.004880,100.898356,54.137193,30.866985,30.719272,26.277428,17.272123,13.726085,10.992130,12.705518,14.660251,8.021843,7.469687,0.005126,1.016201,1.531305e+00,0.918641,1.093630e+00,1.318823,0.735423,0.488321,-6.541902e-01,-0.925812,-6.544207e-01,-7.792018e-01,-0.447860,7.472774e-06,35.670984,23.705415,17.426696,13.158707,9.734639,9.138208,9.333275,7.470029,6.306070,9.371829,7.200817,5.757692,0.001556,-0.129505,0.304341,0.058480,-0.186238,0.038921,0.129330,-0.040977,0.018681,0.078089,-0.198750,-0.178529,-0.300619,-1.747483e-06,25.953198,23.751825,18.101227,13.577461,10.277578,8.971152,10.167631,9.930777,7.134307,10.296205,9.756101,6.965422,0.001012,one
1,-529.008176,95.746875,-15.008879,6.325520,-25.500859,-1.391962,13.729163,-11.764929,-4.405077,-3.205753,-12.717512,-4.035453,0.020440,178.861944,100.796837,37.303266,13.963470,33.536281,15.973280,11.582109,12.901510,6.544613,9.530051,10.995708,8.348934,0.027430,4.536366,-2.015335e-01,-3.585584,4.040602e-01,-0.088948,-0.078999,0.410872,-3.533421e-01,0.118679,1.098207e-01,-8.996757e-02,0.147255,1.022617e-05,55.882945,36.463948,25.534173,10.215644,13.473641,10.003663,6.607842,7.647879,6.678018,7.178864,5.503543,4.907353,0.008091,0.908630,0.574926,-0.897691,0.063177,-0.421624,0.101591,-0.543620,0.522394,0.224634,-0.100589,0.063630,-0.252243,-1.255980e-06,42.816661,39.960212,33.310423,11.802706,13.767670,11.194371,6.752684,8.593841,9.459561,9.543965,7.180339,6.629997,0.005272,one
2,-347.846479,94.967923,-9.343790,4.754978,9.212029,7.082777,-24.783892,7.576551,-12.239119,-5.459896,-1.162680,-4.951098,0.076675,170.389264,78.723045,20.045103,17.515955,33.174565,17.413751,22.462004,12.062866,14.157757,7.792985,9.193893,10.794645,0.090008,0.000000,2.148819e-17,0.000000,-1.146037e-16,0.000000,0.000000,0.000000,1.719055e-16,0.000000,-5.730183e-17,-2.865092e-17,0.000000,-3.359839e-07,49.425675,26.653604,16.721379,10.613685,14.402276,9.576581,7.787498,6.785333,7.852676,4.914879,6.342105,5.883537,0.026805,-0.069822,0.036106,0.069536,-0.082527,-0.006226,0.075505,-0.042447,-0.034329,0.050130,0.000545,-0.033085,0.006105,-2.113945e-05,43.622433,24.584221,19.860864,12.354676,12.111168,12.450000,8.581704,8.245251,8.955527,6.786446,7.983354,6.890580,0.013977,one
3,-518.709180,89.874168,11.233930,19.984826,3.536470,3.724802,-0.895394,11.661922,-11.884202,8.425449,-3.962772,-5.964429,0.026000,180.190323,77.201838,31.980809,15.726209,25.979221,15.669736,8.771908,10.736849,21.501264,12.976462,7.958682,14.971760,0.036308,0.460863,6.191786e-01,0.530669,4.042414e-01,0.254921,0.100178,-0.032804,-1.203931e-01,-0.163776,-1.880945e-01,-2.170961e-01,-0.251061,2.175636e-06,49.820901,27.360206,20.661984,9.619919,11.443960,10.355560,4.849233,5.817271,9.154988,5.467023,5.373065,8.050350,0.011590,0.059099,0.067665,0.033572,0.004187,-0.015470,-0.036476,-0.063528,-0.086565,-0.094014,-0.085762,-0.071576,-0.061458,2.12

In [16]:
# Guardamos nuestro dataframe en un csv
audio_df.to_csv(r'/content/audio_digits.csv')